In [1]:
import numpy as np 
import pandas as pd
from monotonic_binning.monotonic_woe_binning import Binning

In [2]:
training_data = pd.read_csv('https://files.challengerocket.com/files/lions-den-ing-2024/development_sample.csv')
test_data = pd.read_csv('https://files.challengerocket.com/files/lions-den-ing-2024/testing_sample.csv')
training_data['_r_']

0        0.267045
1        0.533040
2        0.861555
3        0.506451
4        0.210913
           ...   
49995    0.783954
49996    0.172590
49997    0.060722
49998    0.845866
49999    0.733623
Name: _r_, Length: 50000, dtype: float64

In [3]:
# średnia wartość _r_ w zależności dla osoby która zbankrutował i niezbankrutowała
print(training_data.groupby(by=['target'])['_r_'].mean())
# procent ludzi który poszedł w default
percent = (training_data['target'].sum()/training_data.shape[0])
print(percent)

target
0.0    0.498032
1.0    0.497357
Name: _r_, dtype: float64
0.02254


In [4]:
# średnia wartość _r_ w zależności dla osoby która zbankrutował i niezbankrutowała
print(test_data.groupby(['target'])['_r_'].mean())
# procent ludzi który poszedł w default
percent = (test_data['target'].sum()/test_data.shape[0])
print(percent)

target
0.0    0.493484
1.0    0.502210
Name: _r_, dtype: float64
0.0242


In [5]:
training_data.columns[5]
#plik z opisem zmiennych
names_xlsx = pd.read_excel('C:\\Users\\ludwi\\OneDrive\\Pulpit\\Lions_den\\variables_description.xlsx')
#Słownik zmian nazw kolumn
names = {f'{names_xlsx['Column'][i]}':f'{names_xlsx['Description'][i]}' for i in range(5, len(names_xlsx))}
#Rename datasetów
training_data = training_data.rename(columns=names)
test_data = test_data.rename(columns = names)

In [6]:
#Czyszczenie Training DataSeta z NaN
print(training_data['target'].isna().sum()/training_data.shape[0])
training_data = training_data.dropna(subset=['target'])
print(training_data)
print('+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
#Czyszczenie Test DataSeta z NaN
print(test_data['target'].isna().sum()/test_data.shape[0])
test_data = test_data.dropna(subset=['target'])
print(test_data)

0.26564
             ID  customer_id   application_date  target Application_status  Number of applicants  Loan purpose Distribution channel  Application amount  Credit duration (months)  Payment frequency  Installment amount  Value of the goods (car)  Application data: income of main applicant  \
0      11034977     32537148  01Feb2010 0:00:00     0.0           Approved                     1           2.0                    1                7800                        99                  1              108.73                       NaN                                        5640   
1      11034978     32761663  01Feb2010 0:00:00     0.0           Approved                     1           1.0                    2               11100                        78                  1              195.99                   11200.0                                        3000   
2      11034979     32701063  01Feb2010 0:00:00     0.0           Approved                     2           3.0            

In [7]:
for i in names.values():
    print(i,'|||',training_data[f'{i}'].isna().sum(),'|||',training_data[f'{i}'].shape[0],'|||',training_data[f'{i}'].isna().sum()/training_data[f'{i}'].shape[0])


Number of applicants ||| 0 ||| 36718 ||| 0.0
Loan purpose ||| 1018 ||| 36718 ||| 0.027724821613377634
Distribution channel ||| 1018 ||| 36718 ||| 0.027724821613377634
Application amount ||| 0 ||| 36718 ||| 0.0
Credit duration (months) ||| 0 ||| 36718 ||| 0.0
Payment frequency ||| 0 ||| 36718 ||| 0.0
Installment amount ||| 0 ||| 36718 ||| 0.0
Value of the goods (car) ||| 20538 ||| 36718 ||| 0.5593441908600686
Application data: income of main applicant ||| 0 ||| 36718 ||| 0.0
Application data: income of second applicant ||| 28043 ||| 36718 ||| 0.7637398551119342
Application data: profession of main applicant ||| 0 ||| 36718 ||| 0.0
Application data: profession of second applicant ||| 28043 ||| 36718 ||| 0.7637398551119342
Application data: employment date (main applicant) ||| 0 ||| 36718 ||| 0.0
Application data: marital status of main applicant ||| 0 ||| 36718 ||| 0.0
Application data: number of children of main applicant ||| 0 ||| 36718 ||| 0.0
Application data: number of dependences o

In [8]:
print(training_data['Clasification of the vehicle (Car, Motorbike)'])

0        NaN
1        1.0
2        NaN
3        0.0
5        NaN
        ... 
49992    NaN
49993    NaN
49995    0.0
49997    1.0
49999    NaN
Name: Clasification of the vehicle (Car, Motorbike), Length: 36718, dtype: float64


In [9]:
# Dodanie Zmiennej Binarnej 'Is There Second Applicant'
training_data['Is There Second Applicant'] = (~training_data['Application data: income of second applicant'].isna()).astype(int)
print(training_data)


             ID  customer_id   application_date  target Application_status  Number of applicants  Loan purpose Distribution channel  Application amount  Credit duration (months)  Payment frequency  Installment amount  Value of the goods (car)  Application data: income of main applicant  \
0      11034977     32537148  01Feb2010 0:00:00     0.0           Approved                     1           2.0                    1                7800                        99                  1              108.73                       NaN                                        5640   
1      11034978     32761663  01Feb2010 0:00:00     0.0           Approved                     1           1.0                    2               11100                        78                  1              195.99                   11200.0                                        3000   
2      11034979     32701063  01Feb2010 0:00:00     0.0           Approved                     2           3.0                    

In [11]:
bin_object = Binning('target', n_threshold = 50, y_threshold = 10, p_threshold = 0.35, sign=False)
bin_object.fit(training_data)

In [13]:
print(bin_object.woe_summary)
print('+++++++++++++++++++++++')
print(bin_object.bins)

         ID  nsamples     means   bads    goods  dist_good  dist_bad    WOE_ID  IV_components    ID_shift                 labels
0  11084976    3680.0  0.033696  124.0   3556.0   0.099776  0.110027 -0.097799       0.001003        -inf        -inf-11084976.0
1  11079906    5836.0  0.032385  189.0   5647.0   0.158446  0.167702 -0.056777       0.000526  11084976.0  11079906.0-11084976.0
2  11071909    1768.0  0.031674   56.0   1712.0   0.048036  0.049689 -0.033844       0.000056  11079906.0  11071909.0-11079906.0
3  11069443   21321.0  0.030111  642.0  20679.0   0.580219  0.569654  0.018376       0.000194  11071909.0  11069443.0-11071909.0
4  11040471    2947.0  0.029522   87.0   2860.0   0.080247  0.077196  0.038759       0.000118  11069443.0  11040471.0-11069443.0
5  11036508     644.0  0.026398   17.0    627.0   0.017593  0.015084  0.153824       0.000386  11040471.0  11036508.0-11040471.0
6  11035660     571.0  0.021016   12.0    559.0   0.015685  0.010648  0.387333       0.001951  11

In [17]:
train_transform = bin_object.transform(training_data)
train_transform

,ID,customer_id,application_date,target,Application_status,Number of applicants,Loan purpose,Distribution channel,Application amount,Credit duration (months),Payment frequency,Installment amount,Value of the goods (car),Application data: income of main applicant,Application data: income of second applicant,Application data: profession of main applicant,Application data: profession of second applicant,Application data: employment date (main applicant),Application data: marital status of main applicant,Application data: number of children of main applicant,Application data: number of dependences of main applicant,Spendings estimation,Property ownership for property renovation,"Clasification of the vehicle (Car, Motorbike)",Number of requests during the last 3 months (External data),Number of requests during the last 6 months (External data),Number of requests during the last 9 months (External data),Number of requests during the last 12 months (External data),Limit on credit card,Amount on current account,Amount on savings account,Arrear in last 3 months (indicator),Arrear in last 12 months (indicator),Credit bureau score (Exterval data),Average income (Exterval data),_r_,Is There Second Applicant,bins,ID_bins
0,11034977,32537148,01Feb2010 0:00:00,0.0,Approved,1,2.0,1,7800,99,1,108.73,NaN,5640,NaN,5,NaN,26Nov2004,2,0,0,3272.31,1.0,NaN,1,1,1,1,0,6768.42,0.00,0,0,0,3899,0.267045,0,"(-inf, 11035660.0]","(-inf, 11035660.0]"
1,11034978,32761663,01Feb2010 0:00:00,0.0,Approved,1,1.0,2,11100,78,1,195.99,11200.0,3000,NaN,2,NaN,16Sep1996,4,4,4,1387.35,NaN,1.0,0,0,0,0,15000,5937.66,3870.60,0,0,0,3899,0.533040,0,"(-inf, 11035660.0]","(-inf, 11035660.0]"
2,11034979,32701063,01Feb2010 0:00:00,0.0,Approved,2,3.0,1,2400,15,1,248.34,NaN,6240,4300.0,5,5.0,06May1989,1,0,0,4703.28,NaN,NaN,1,2,2,2,0,5647.77,1463.08,0,0,10,3899,0.861555,1,"(-inf, 11035660.0]","(-inf, 11035660.0]"
3,11034980,32386786,01Feb2010 0:00:00,0.0,Approved,3,1.0,2,11800,30,6,3538.68,12400.0,4560,2500.0,6,5.0,06Dec1987,1,0,0,2508.70,NaN,0.0,0,0,0,0,0,3594.90,21563.78,0,0,10,3899,0.506451,1,"(-inf, 11035660.0]","(-inf, 11035660.0]"
5,11034982,32861919,02Feb2010 0:00:00,0.0,Approved,1,2.0,1,7200,36,1,267.19,NaN,3960,NaN,5,NaN,31Dec9999,0,1,1,2034.22,0.0,NaN,0,1,2,2,0,1828.62,NaN,0,0,10,3899,0.003446,0,"(-inf, 11035660.0]","(-inf, 11035660.0]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49992,11084969,32845249,31Dec2020 0:00:00,0.0,Approved,1,3.0,3,5800,6,6,12180.00,NaN,21120,NaN,1,NaN,28Jun2003,0,0,0,7458.44,NaN,NaN,1,3,3,3,0,NaN,NaN,0,0,0,12832,0.654447,0,"(11079906.0, 11084976.0]","(11079906.0, 11084976.0]"
49993,11084970,32623752,31Dec2020 0:00:00,0.0,Approved,1,2.0,2,26100,15,3,7342.23,NaN,16680,NaN,4,NaN,25Oct2001,1,1,1,5872.11,1.0,NaN,0,0,0,0,0,10816.79,108005.34,0,0,0,12832,0.225416,0,"(11079906.0, 11084976.0]","(11079906.0, 11084976.0]"
49995,11084972,32554904,31Dec2020 0:00:00,1.0,Approved,2,1.0,1,49200,111,1,621.69,52600.0,6240,11900.0,4,5.0,28Mar2008,0,0,1,6829.92,NaN,0.0,1,2,2,2,0,NaN,NaN,0,0,0,12832,0.783954,1,"(11079906.0, 11084976.0]","(11079906.0, 11084976.0]"
49997,11084974,32433438,31Dec2020 0:00:00,0.0,Approved,1,1.0,1,46700,18,1,3963.94,79000.0,24840,NaN,5,NaN,28Oct2006,0,0,0,9876.73,NaN,1.0,0,0,0,0,0,9288.87,255018.31,0,0,0,12832,0.060722,0,"(11079906.0, 11084976.0]","(11079906.0, 11084976.0]"
